In [1]:
import argparse
import random
import re
from pathlib import Path

import dspy
from dspy.evaluate import Evaluate

from ccf.pylib import trait_extractor as te

In [2]:
args = argparse.Namespace(
    examples_json=Path("../data/fna/Asteraceae_examples_2025-03-19c.json"),
    # model="ollama_chat/gemma3:27b",
    model="ollama_chat/tinyllama",
    api_base="http://localhost:11434",
    api_key="",
    train_split=0.1,
    dev_split=0.5,
    test_split=0.4,
    seed=2203673,
)

In [3]:
random.seed(args.seed)

In [4]:
examples = te.read_examples(args.examples_json)
dataset = te.split_examples(examples, args.train_split, args.dev_split)

lm = dspy.LM(args.model, api_base=args.api_base, api_key=args.api_key, cache=False)
dspy.configure(lm=lm)

trait_extractor = dspy.Predict(te.TraitExtractor)
# trait_extractor = dspy.ChainOfThought(te.TraitExtractor)

In [5]:
evaluator = Evaluate(
    devset=dataset["dev"],
    metric=te.score_prediction,
    num_threads=1,
    display_progress=True,
    # display_table=True,
    provide_traceback=True,
)

In [6]:
evaluator(trait_extractor, devset=dataset["dev"])

Average Metric: 37.93 / 203 (18.7%):  74%|████████████████████████████████████████████████████████████████████████▏                        | 151/203 [05:39<02:13,  2.56s/it]

2025/03/26 00:58:01 ERROR dspy.utils.parallelizer: Error processing item Example({'family': 'Asteraceae', 'taxon': 'Eclipta prostrata', 'text': '<b>Leaf </b>blades 2–10 cm × 4–30+ mm. <b>Ray</b> laminae ca. 2 mm. <b>Disc</b> corollas ca. 1.5 mm. <b>Cypselae</b> ca. 2.5 mm. <b>2n</b> = 22 (as E. alba).\nPhenology: Flowering year round, mostly summer–fall.\nHabitat: Wet places, often ruderal\nElevation: 60–1600 m', 'prompt': '\n    What is the plant size,\n    leaf shape, leaf length, leaf width, leaf thickness,\n    seed length, seed width,\n    fruit type, fruit length, fruit width,\n    deciduousness, phenology, habitat, elevation?\n    If it is not mentioned return an empty value.\n    ', 'plant_height': '', 'leaf_shape': '', 'leaf_length': '2–10 cm', 'leaf_width': '4–30+ mm', 'leaf_thickness': '', 'fruit_type': 'Cypselae', 'fruit_length': '2.5 mm', 'fruit_width': '', 'seed_length': '', 'seed_width': '', 'deciduousness': '', 'phenology': 'Flowering year round, mostly summer–fall', 'h

Average Metric: 49.93 / 203 (24.6%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [07:37<00:00,  2.26s/it]

2025/03/26 00:59:58 INFO dspy.evaluate.evaluate: Average Metric: 49.925144404796285 / 203 (24.6%)


24.59

In [7]:
mipro_optimizer = dspy.MIPROv2(
    metric=te.score_prediction,
    auto="medium",
)

optimized_trait_extractor = mipro_optimizer.compile(
    trait_extractor,
    trainset=dataset["train"],
    max_bootstrapped_demos=4,
    requires_permission_to_run=False,
    minibatch=False,
)

2025/03/26 00:59:58 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 32

2025/03/26 00:59:58 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/26 00:59:58 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/26 00:59:58 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 44%|█████████████████████████████████████████████████████████████▎                                                                            | 4/9 [00:09<00:12,  2.40s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/19


 44%|█████████████████████████████████████████████████████████████▎                                                                            | 4/9 [00:05<00:06,  1.39s/it]


...

2025/03/26 01:27:07 INFO dspy.evaluate.evaluate: Average Metric: 18.47314039189231 / 32 (57.7%)
2025/03/26 01:27:07 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 57.73 with parameters ['Predictor 0: Instruction 6', 'Predictor 0: Few-Shot Set 14'].
2025/03/26 01:27:07 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [26.49, 61.29, 57.93, 57.93, 67.8, 62.22, 56.02, 66.1, 52.84, 68.93, 68.86, 67.66, 62.88, 69.6, 56.24, 70.11, 70.24, 69.65, 69.76, 57.54, 66.53, 70.23, 57.73]
2025/03/26 01:27:07 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 70.24
2025/03/26 01:27:07 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/26 01:27:07 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 24 / 25 =====



Average Metric: 22.45 / 32 (70.1%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:52<00:00,  1.63s/it]

2025/03/26 01:27:59 INFO dspy.evaluate.evaluate: Average Metric: 22.445593759594647 / 32 (70.1%)
2025/03/26 01:27:59 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 70.14 with parameters ['Predictor 0: Instruction 13', 'Predictor 0: Few-Shot Set 4'].
2025/03/26 01:27:59 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [26.49, 61.29, 57.93, 57.93, 67.8, 62.22, 56.02, 66.1, 52.84, 68.93, 68.86, 67.66, 62.88, 69.6, 56.24, 70.11, 70.24, 69.65, 69.76, 57.54, 66.53, 70.23, 57.73, 70.14]
2025/03/26 01:27:59 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 70.24
2025/03/26 01:27:59 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/26 01:27:59 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====



Average Metric: 20.83 / 32 (65.1%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:02<00:00,  1.94s/it]

2025/03/26 01:29:02 INFO dspy.evaluate.evaluate: Average Metric: 20.834369872746848 / 32 (65.1%)
2025/03/26 01:29:02 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 65.11 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 10'].
2025/03/26 01:29:02 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [26.49, 61.29, 57.93, 57.93, 67.8, 62.22, 56.02, 66.1, 52.84, 68.93, 68.86, 67.66, 62.88, 69.6, 56.24, 70.11, 70.24, 69.65, 69.76, 57.54, 66.53, 70.23, 57.73, 70.14, 65.11]
2025/03/26 01:29:02 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 70.24
2025/03/26 01:29:02 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/26 01:29:02 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 70.24!


In [8]:
evaluator(optimized_trait_extractor, devset=dataset["dev"])

Average Metric: 16.60 / 203 (8.2%):  12%|████████████▏                                                                                      | 25/203 [01:12<10:10,  3.43s/it]

2025/03/26 01:30:19 ERROR dspy.utils.parallelizer: Error processing item Example({'family': 'Asteraceae', 'taxon': 'Glebionis coronaria', 'text': '<b>Leaf </b>blades oblong to obovate, mostly 30–55+ × 15–30+ mm, mostly 2–3-pinnately lobed, ultimate margins dentate. <b>Ray</b> corollas pale yellow, sometimes white-tipped, laminae oblong to linear, 15–25 mm. <b>Ray</b> cypselae triquetrous, 2.5–3 mm, angles ± winged, faces obscurely nerved or ribbed; disc cypselae ± prismatic, ± compressed, 2.5–3 mm, obscurely ribbed, sometimes with adaxial (rarely the abaxial) rib ± winged. <b>2n</b> = 18.\nPhenology: Flowering Apr–Jul.\nHabitat: Disturbed sites\nElevation: 0–200 m', 'prompt': '\n    What is the plant size,\n    leaf shape, leaf length, leaf width, leaf thickness,\n    seed length, seed width,\n    fruit type, fruit length, fruit width,\n    deciduousness, phenology, habitat, elevation?\n    If it is not mentioned return an empty value.\n    ', 'plant_height': '', 'leaf_shape': 'oblong 

Average Metric: 135.89 / 203 (66.9%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [09:19<00:00,  2.76s/it]

2025/03/26 01:38:21 INFO dspy.evaluate.evaluate: Average Metric: 135.88899635063956 / 203 (66.9%)


66.94

In [9]:
dspy.inspect_history(n=1)





[2025-03-26T01:38:21.811068]

System message:

Your input fields are:
1. `family` (str): The family taxon
2. `taxon` (str): The taxon
3. `text` (str): The species description text
4. `prompt` (str): Extract these traits

Your output fields are:
1. `plant_height` (str): The plant height
2. `leaf_shape` (str): The leaf shape
3. `leaf_length` (str): The leaf length
4. `leaf_width` (str): The leaf width
5. `leaf_thickness` (str): The leaf thickness
6. `fruit_type` (str): The fruit type
7. `fruit_length` (str): The fruit length
8. `fruit_width` (str): The fruit width
9. `seed_length` (str): The seed length
10. `seed_width` (str): The seed witdth
11. `deciduousness` (str): The plant's deciduousness
12. `phenology` (str): The phenology
13. `habitat` (str): The habitat
14. `elevation` (str): The elevation

All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## family ## ]]
{family}

[[ ## taxon ## ]

In [10]:
ex = args.examples_json.stem.replace("_examples", "")
mod = args.model.split("/")[-1]
mod = re.sub(r"\W", "_", mod)
optimized_trait_extractor.save(f"../data/fna/optimized_{mod}_trait_extractor_{ex}.json")